In [7]:
import requests
import json
import json
import requests
from bs4 import BeautifulSoup
import os


# create the directory if it doesn't already exist
if not os.path.exists('News-Articles'):
    os.makedirs('News-Articles')


# Load URLs from JSON file
with open('urls.json', 'r') as f:
    urls = json.load(f)

for url in urls:
    url_string = url["url"]
    r = requests.get(url_string)

    if r.status_code == 200:
        soup = BeautifulSoup(r.content, "html.parser")
        article = soup.find("article")
        # Get the text out of the soup and print it
        content = article.get_text()
        # print(content)
        
        # construct the path to the file within the News-Articles directory
        filename = os.path.join('News-Articles', url['title'] + '.txt')
        with open(filename, 'a') as f:
            f.write(content)
            f.write('\n\n')
        print(f'Successfully saved {filename}')
    else:
        print(f"Error: {r.status_code}")


Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/business.txt
Successfully saved News-Articles/business.txt
Successfully saved News-Articles/business.txt
Successfully saved News-Articles/technology.txt
Successfully saved News-Articles/technology.txt
Successfully saved News-Articles/technology.txt


In [8]:
import csv

# Directory where the .txt files are located
directory = './News-Articles/'

# Create a list to store the rows of the CSV file
csv_rows = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Read the contents of the file
        with open(os.path.join(directory, filename), 'r') as f:
            content = f.read().replace('\n', ' ')
        
        # Get the title from the filename
        title = filename.split('.')[0]
        
        # Append the title and content to the CSV row list
        csv_rows.append([title, content])

# Write the CSV file
with open('news_corpus.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Topic', 'Content'])  # write the header row
    writer.writerows(csv_rows)  # write the content rows
    print("Successfully created news_corpus.csv")


Successfully created news_corpus.csv


In [17]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load the news corpus into a Pandas DataFrame
df = pd.read_csv('news_corpus.csv')

# Initialize NLTK components
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Function to preprocess text data
def preprocess_text(Content):
    # Remove punctuation
    Content = Content.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the text into words
    words = word_tokenize(Content.lower())

    # Remove stop words
    words = [w for w in words if not w in stop_words]

    # Lemmatize the remaining words
    words = [lemmatizer.lemmatize(w) for w in words]

    # Rejoin the words into a single string
    return ' '.join(words)

# Apply the preprocessing function to the news corpus
df['Content'] = df['Content'].apply(preprocess_text)
print(df)

        Topic                                            Content
0    business  happening house price could crashpublished7 ho...
1  technology  elon musk among expert urging halt ai training...
2      sports  manchester united premier league club owe almo...


[nltk_data] Downloading package wordnet to /home/silver/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF Vectorizer object
vectorizer = TfidfVectorizer()

# Vectorize the text data
X = vectorizer.fit_transform(df['Content'])

# Get the labels
y = df['Topic']
